# Create Latex Tables with Enemy Wave Data

Create a bunch of latex tables containing the wave data.

## Extract the level data from the source files

In [1]:
"""
Pull out all the level data from the source files.
"""
files = ["../iridisalpha/src/level_data/level_data.asm",
         "../iridisalpha/src/level_data/level_data2.asm"]
d = {}
level_data = {}
ch = None
ck = None
for f in files:
    ld = open(f, 'r')
    while True:
        l = ld.readline()
        if not l:
            break
        if "pieceOfPlanet" in l:
            continue
        if l[:2].strip():
            if ch and d:
                level_data[ch] = d
            ch = l.split()[0].strip()
            d = {}
            continue

        k = l[10:17]
        if "Byte" in k:
            ck = k
            continue

        if "BYTE" in l:
            v = l[14:].strip().split(',')[0].replace('<','')
            d[ck] = v

# Drop any spurious entries
level_data = {k:c for k,c in level_data.items() if len(c) == 30}
len(level_data.keys())

200

In [2]:
level_data

{'planet4Level19Additional': {'Byte 0 ': '$09',
  'Byte 2 ': 'LOZENGE',
  'Byte 3 ': '$00',
  'Byte 5 ': 'LOZENGE',
  'Byte 6 ': '$00',
  'Byte 8 ': 'nullPtr',
  'Byte 10': 'nullPtr',
  'Byte 11': '$00',
  'Byte 13': 'nullPtr',
  'Byte 14': '$00',
  'Byte 15': '$20',
  'Byte 17': 'planet4Level19Additional2ndWave',
  'Byte 18': '$00',
  'Byte 19': '$23',
  'Byte 20': '$02',
  'Byte 21': '$02',
  'Byte 22': '$01',
  'Byte 23': '$23',
  'Byte 25': 'nullPtr',
  'Byte 27': 'nullPtr',
  'Byte 29': 'planet4Level19Additional2ndWave',
  'Byte 31': 'planet4Level19Additional2ndWave',
  'Byte 32': '$00',
  'Byte 33': '$00',
  'Byte 34': '$00',
  'Byte 35': '$02',
  'Byte 36': '$00',
  'Byte 37': '$00',
  'Byte 38': '$00',
  'Byte 39': '$00'},
 'planet4Level19Additional2ndWave': {'Byte 0 ': '$05',
  'Byte 2 ': 'SPINNING_RING1',
  'Byte 3 ': '$01',
  'Byte 5 ': 'SPINNING_RING1',
  'Byte 6 ': '$00',
  'Byte 8 ': 'nullPtr',
  'Byte 10': 'nullPtr',
  'Byte 11': '$00',
  'Byte 13': 'nullPtr',
  'Byte 14

## Look at unused data

In [19]:
header = """
\\begin{figure}[H]
  {
  \\setlength{\\tabcolsep}{3.0pt}
  \\setlength\\cmidrulewidth{\\heavyrulewidth} % Make cmidrule = 
  \\begin{adjustbox}{width=\\textwidth}
"""

footer = """
  \\end{adjustbox}

  }\\caption*{%CAPTION%.}
\\end{figure}
"""

byte_descriptions = {
"Level" : "",
"Byte 0":"Index into array for sprite color",                                   
"Byte 1":"First sprite value for the attack ship on the upper planet",          
"Byte 2":"Last sprite value for the attack ship on the upper planet",           
"Byte 3":"The animation frame rate for the attack ship.",                       
"Byte 4":"First sprite value for the attack ship on the lower planet",          
"Byte 5":"Last sprite value for the attack ship on the lower planet",           
"Byte 6":"Rate at which to switch to alternate enemy mode.",                    
"Byte 7":"Lo Ptr for alternate enemy mode",                                     
"Byte 8":"Hi Ptr for alternate enemy mode",                                     
"Byte 9":"textbf{Unused} Lo Ptr to an arbitrary run of Bytes 18-21.",
"Byte 10":"textbf{Unused} Hi Ptr to an arbitrary run of Bytes 18-21.",
"Byte 11":"textbf{Unused} Rate limite for use of Bytes 9 and 10.",              
"Byte 12":"textbf{Unused}",                                                     
"Byte 13":"textbf{Unused}",                                                     
"Byte 14":"Controls the rate at which new enemies are added.",
"Byte 15":"Update rate for switching to the level data in Bytes 16/17",                                        
"Byte 16":"Lo Ptr to wave data we switch to periodically.",               
"Byte 17":"Hi Ptr to wave data we switch to periodically.",               
"Byte 18":"X Pos movement for attack ship.",                                    
"Byte 19":"Y Pos movement pattern for attack ship.",                            
"Byte 20":"X Pos Frame Rate for Attack ship.",                                  
"Byte 21":"Y Pos Frame Rate for Attack ship.",                                  
"Byte 22":"Stickiness factor, does the enemy stick to the player",              
"Byte 23":"Does the enemy gravitate quickly toward the player when its hit?",   
"Byte 24":"Lo Ptr for second wave of attack ships.",                            
"Byte 25":"Hi Ptr for second wave of attack ships.",                            
"Byte 26":"Lo Ptr for third wave of attack ships.",                             
"Byte 27":"Hi Ptr for third wave of attack ships.",                             
"Byte 28":"Lo Ptr for Explosion animation.",                                    
"Byte 29":"Hi Ptr for Explosion animation.",                                    
"Byte 30":"Lo Ptr for another set of wave data for this level.",                
"Byte 31":"Hi Ptr for another set of wave data for this level.",                
"Byte 32":"Lo Ptr for fourth wave of attack ships.",                            
"Byte 33":"Hi Ptr for fourth wave of attack ships.",                            
"Byte 34":"Points for hitting the enemy.",                                      
"Byte 35":"Energy increase multiplier for hitting an enemy.",
"Byte 36":"Is the ship a spinning ring, i.e. does it allow the gilby to warp?", 
"Byte 37":"Number of waves in data.",                                           
"Byte 38":"Number of ships in wave.",                                           
"Byte 39":"textbf{Unused} byte.",                                               
}
adl = "\\addlinespace"
leg = "\\multicolumn{%len%}{@{}l@{}}{%BYTE%}\\\\"


In [4]:
import pandas as pd
from tabulate import tabulate
from texttable import Texttable
import latextable


styles = [dict(selector="caption", 
    props=[("text-align", "center"),
    ("font-size", "120%"),
    ("color", 'black')])]

def createTable(indices, cands):
    """ Make a table and column list of the wave data"""

    tb = []
    cols = ['Planet', 'Level'] 
    all_cols = list(list(cands.values())[0].keys())
    for i in indices:
        cols += [all_cols[i]]
    for p,d in cands.items():
        t = p.replace('planet','').replace('Level', ' ').replace('Data', '')
        f = [*t.split()]
        l = list(d.values())
        for i in indices:
            if i >= len(l):
                continue
            f += [l[i]]
        tb += [f]
    level_table = sorted(tb, key=lambda x: (x[0],int(x[1])))
    return (level_table, cols)


In [10]:
# Restrict `cands` to just the guys that are the initial wave data for each
# wave. We'll look at the others separately.
cands = {c:v for c,v in sorted(level_data.items())
         if c.startswith("planet") and c.endswith("Data")}

all_fields = list(range(0,29))
#all_fields = [0,2,4,7,9,10,12,13,14,15]
level_table, cols = createTable(all_fields, cands)

# Get the unique values used in the initial level data.
level_fields = [l[2:] for l in level_table]


In [20]:
pointer_data = [2,25,27]

level_table, cols = createTable(pointer_data, cands)
for i in range(1,6):
    lt = [l[1:] for l in level_table if l[0] == str(i)]
    print(header)
    
    ltable = tabulate(lt, headers=cols[1:], tablefmt='latex_booktabs')
    leg_cols = [f"{c}: {byte_descriptions[c.strip()]}" for c in cols[2:]]
    legend = '\n'.join([leg.replace("%len%", str(len(leg_cols)))
                        .replace("%BYTE%", lc) for lc in leg_cols])
    ltable = ltable.replace("\\bottomrule", "\\addlinespace\n\\bottomrule\n" + legend)

    print(ltable)
    print(footer.replace("%CAPTION%", f"Planet {i} - Pointer Data"))


\begin{figure}[H]
  {
  \setlength{\tabcolsep}{3.0pt}
  \setlength\cmidrulewidth{\heavyrulewidth} % Make cmidrule = 
  \begin{adjustbox}{width=\textwidth}

\begin{tabular}{rlll}
\toprule
   Level & Byte 3    & Byte 35   & Byte 37   \\
\midrule
       1 & \$03       & \$02       & \$04       \\
       2 & \$01       & \$01       & \$04       \\
       3 & \$04       & \$01       & \$04       \\
       4 & \$03       & \$02       & \$04       \\
       5 & \$00       & \$05       & \$04       \\
       6 & \$03       & \$04       & \$04       \\
       7 & \$00       & \$03       & \$04       \\
       8 & \$03       & \$02       & \$04       \\
       9 & \$03       & \$08       & \$04       \\
      10 & \$00       & \$06       & \$04       \\
      11 & \$06       & \$05       & \$04       \\
      12 & \$02       & \$02       & \$04       \\
      13 & \$04       & \$02       & \$04       \\
      14 & \$05       & \$08       & \$04       \\
      15 & \$00       & \$03       & \$04